In [1]:
#用一個 rule做推薦：按持有人數將產品排序，再依序推薦給顧客
#source: https://www.kaggle.com/sudalairajkumar/simple-btb
# Private score: 0.0191288; Public score: 0.018968

import csv
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#選取資料
print("Reading files and getting top products..")

dtype_dict = {'ind_cco_fin_ult1': 'float16', 'ind_deme_fin_ult1': 'float16', 'ind_aval_fin_ult1': 'float16', 'ind_valo_fin_ult1': 'float16', 'ind_reca_fin_ult1': 'float16', 'ind_ctju_fin_ult1': 'float16', 'ind_cder_fin_ult1': 'float16', 'ind_plan_fin_ult1': 'float16', 'ind_fond_fin_ult1': 'float16', 'ind_hip_fin_ult1': 'float16', 'ind_pres_fin_ult1': 'float16', 'ind_nomina_ult1': 'float16', 'ind_cno_fin_ult1': 'float16', 'ncodpers': 'int64', 'ind_ctpp_fin_ult1': 'float16', 'ind_ahor_fin_ult1': 'float16', 'ind_dela_fin_ult1': 'float16', 'ind_ecue_fin_ult1': 'float16', 'ind_nom_pens_ult1': 'float16', 'ind_recibo_ult1': 'float16', 'ind_deco_fin_ult1': 'float16', 'ind_tjcr_fin_ult1': 'float16', 'ind_ctop_fin_ult1': 'float16', 'ind_viv_fin_ult1': 'float16', 'ind_ctma_fin_ult1': 'float16'}
train = pd.read_csv("train_ver2.csv", dtype=dtype_dict, usecols=['ncodpers', 'ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1'])

#製造一個(產品, 持有人數)的dictionary
count_dict = {}
for col_name in list(train.columns):
    if col_name != 'ncodpers':
        count_dict[col_name] = np.sum(train[col_name].astype('float64'))

#依持有數排序產品        
top_products = sorted(count_dict, key=count_dict.get, reverse=True)

#只留下最後一個月份的顧客，因為我們最後要排除該顧客最後一個月已持有的產品
# You will predict what a customer will buy in addition to what they already had at 2016-05-28
print("Drop duplicates and keep last one")
#train['ncodpers'] = train['ncodpers'].astype('int')
train = train.drop_duplicates('ncodpers', keep='last')

#將submission資料讀入，並left join剛剛讀入的train
print("Read sample submission and merge with train..")
sub = pd.read_csv("sample_submission.csv")
sub = sub.merge(train, on='ncodpers', how='left')
del train
sub.fillna(0, inplace=True)

#開始利用設定好的rule進行推薦
print("Get the top products which are not already bought..")
ofile = open("simple_btb_v2.0.csv","w")
writer = csv.writer(ofile)
writer.writerow(['ncodpers', 'added_products'])
for ind, row in sub.iterrows():
    cust_id = row['ncodpers']
    top7_products = [] 
    for product in top_products:
        if int(row[product]) == 0:
            top7_products.append(str(product))
            if len(top7_products) == 7:
                break
    writer.writerow([cust_id, " ".join(top7_products)])
ofile.close()

Reading files and getting top products..
Drop duplicates and keep last one
Read sample submission and merge with train..
Get the top products which are not already bought..


In [2]:
sub.head()

,ncodpers,added_products,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,ind_tjcr_fin_ult1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,15890,ind_tjcr_fin_ult1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2,15892,ind_tjcr_fin_ult1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
3,15893,ind_tjcr_fin_ult1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,15894,ind_tjcr_fin_ult1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [3]:
print top_products

['ind_cco_fin_ult1', 'ind_ctop_fin_ult1', 'ind_recibo_ult1', 'ind_ecue_fin_ult1', 'ind_cno_fin_ult1', 'ind_nom_pens_ult1', 'ind_nomina_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_dela_fin_ult1', 'ind_valo_fin_ult1', 'ind_fond_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctju_fin_ult1', 'ind_plan_fin_ult1', 'ind_hip_fin_ult1', 'ind_viv_fin_ult1', 'ind_pres_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_cder_fin_ult1', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1']
